In [94]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma
import scipy as sp

In [95]:
class OneToOneITU():
    def __init__(self, n, m, parameters , lbs=(0, 0), tol=1e-9):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
        self.tol = tol 
       
        self.A_ij = parameters[0]
        self.B_ij = parameters[1]

    def D_ij(self, U_i, V_j):
        return (U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij)/(1+ self.A_ij)

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            return self.B_ij[i_idx] - self.A_ij[i_idx] * V_j[None, :]
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, j_idx]

    def get_V_ij(self,i_idx, j_idx , U_i):
        return (self.B_ij[i_idx,j_idx] -  U_i)/ self.A_ij[i_idx,j_idx]

In [96]:
def ITU_parallel_auction(self):

    V_j = np.ones(self.m) * self.lb_V
    mu_ij = np.ones( self.n, dtype= int) * (self.m +1)

    unassigned = np.where(mu_ij == self.m + 1)[0] 
    n_unassigned = len(unassigned)
    iter = 0

    while n_unassigned > 0:# and iter <500000:
        U_ij = self.get_U_ij( V_j, unassigned)
        perm_unmatched = np.all(U_ij <= self.lb_U, axis = 1)

        if np.any(perm_unmatched):
            mu_ij[unassigned] = self.m * perm_unmatched
        
        else:
            U_ij = np.concatenate((U_ij, self.lb_U * np.ones((n_unassigned,1))), axis = 1)
            U_ij_sorted_id = np.argsort(U_ij, axis = 1)
            j_i = U_ij_sorted_id[:,-1]
            w_i = U_ij[np.arange(n_unassigned),U_ij_sorted_id[:,-2] ]
      
            j_unique = np.unique(j_i)
            offers = w_i[:,None] * ( j_i[:,None] == j_unique[None,:])  - 1e22 *  ( j_i[:,None] != j_unique[None,:]) 
            i_j_among_unass = np.argmax(offers, axis = 0)
            i_j = unassigned[ i_j_among_unass]
            best_offer_j = np.max(offers, axis = 0)
            
            mu_ij[  np.any( mu_ij[:,None] == j_unique , axis = 1) ] = self.m +1
            mu_ij[i_j] = j_unique
            V_j[j_unique] += np.maximum(self.get_V_ij(i_j,j_unique, best_offer_j) - V_j[j_unique], self.tol)
       
            
        unassigned = np.where(mu_ij == self.m + 1)[0]
        n_unassigned = len(unassigned)
        iter += 1

    
    matched_i =  mu_ij < self.m
    U_i = np.ones(self.n) * self.lb_U
    U_i[matched_i] = self.get_U_ij(V_j, matched_i)[np.arange(matched_i.sum()), mu_ij[matched_i]]
    mu_ij = mu_ij[:,None] == np.arange(self.m +1)
    
    

    return mu_ij, U_i, V_j


OneToOneITU.ITU_parallel_auction = ITU_parallel_auction

Generate a a parametrized example

In [97]:
np.random.seed(10)
n_i = 3000
m_j = 3400

θ_true =  np.array([.5,.5,5,5])
k = len(θ_true)
X_ijk = np.random.normal(1,1, size = [n_i,m_j, k])
 

A = np.exp( X_ijk[:,:,:2] @ θ_true[:2])
B = X_ijk[:,:,2:4] @ θ_true[2:4]

In [98]:
B.mean()

10.002902857122965

In [99]:
true_mkt = OneToOneITU( n_i,m_j, (A,B), tol= .0001)

In [100]:
mu_obs, u_true, v_true = true_mkt.ITU_parallel_auction()

In [101]:
np.all(u_true[:,None] + A * v_true[None,:] -B )>= 0

True

In [102]:
np.sum(np.maximum(B - u_true[:,None] - A * v_true[None,:] ,0))

4.3469158395487834e-05

# Estimation

In [109]:
def Q(self,θ,mu_obs,X_ijk):
    A_θ = np.exp( X_ijk[:,:,:2] @ θ[:2])
    B_θ = X_ijk[:,:,2:4] @ θ[2:4]

    mkt_θ = OneToOneITU( n_i,m_j, (A_θ,B_θ), tol= .0001)
    mu_θ, u_θ, v_θ = mkt_θ.ITU_parallel_auction()


    Q_θ = ( (mu_obs[:,:-1] * mkt_θ.D_ij(u_θ, v_θ) ).sum() + 
            (mu_obs[:,-1] * u_θ ).sum() + 
            ((mu_obs[:,:-1].sum(0) == 0) * v_θ).sum() +
            np.log( mu_θ.sum()) +
            ((v_true - v_θ)**2).sum())
    


    print(Q_θ)
    print( θ.round(3))
    return Q_θ

OneToOneITU.Q = Q

In [110]:
true_mkt.Q(np.array([.5,.5,5,5]), mu_obs, X_ijk)

8.006367567650246
[0.5 0.5 5.  5. ]


8.006367567650246

In [111]:
def maximum_score_estimator(self,θ_0,mu_obs,X_ijk ):

    problem = sp.optimize.minimize( lambda θ: self.Q(θ,mu_obs,X_ijk)  ,θ_0)  

    return problem.x


OneToOneITU.maximum_score_estimator = maximum_score_estimator

In [113]:
true_mkt.maximum_score_estimator( np.array([1,1,1,1]), mu_obs,X_ijk )

5502.805783408055
[1. 1. 1. 1.]
5502.805775062389
[1. 1. 1. 1.]
5502.80577388004
[1. 1. 1. 1.]
5502.805748108848
[1. 1. 1. 1.]
5502.805777819068
[1. 1. 1. 1.]
9611.885408802475
[1.222 1.254 1.94  1.149]
9611.885650163014
[1.222 1.254 1.94  1.149]
9611.885609612258
[1.222 1.254 1.94  1.149]
9611.885650737515
[1.222 1.254 1.94  1.149]
9611.885110783776
[1.222 1.254 1.94  1.149]
5159.910544587265
[1.066 1.075 1.277 1.044]
5159.910543207974
[1.066 1.075 1.277 1.044]
5159.910536516941
[1.066 1.075 1.277 1.044]
5159.910565979321
[1.066 1.075 1.277 1.044]
5159.910482814618
[1.066 1.075 1.277 1.044]
4631.679052676082
[1.135 1.173 1.491 1.27 ]
4631.679051408337
[1.135 1.173 1.491 1.27 ]
4631.679052553794
[1.135 1.173 1.491 1.27 ]
4631.679074854901
[1.135 1.173 1.491 1.27 ]
4631.679001095115
[1.135 1.173 1.491 1.27 ]
6437.7158283996
[1.468 0.47  1.972 1.779]
6437.716078306277
[1.468 0.47  1.972 1.779]
6437.7157519565035
[1.468 0.47  1.972 1.779]
6437.716037713446
[1.468 0.47  1.972 1.779]
6437.7

array([0.50123215, 0.49642811, 5.00217379, 5.0109778 ])